In [209]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import seaborn as sns

In [210]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [211]:
caminho_arquivo_excel = '/content/drive/MyDrive/ProjetoVeins/ModeloRegressaoLinear/dataset80vh (1).xlsx'
df = pd.read_excel(caminho_arquivo_excel)
print(df.head())

   ID  Posicao  Velocidade
0  16        0           0
1  16       30           1
2  16       31           3
3  16       35           5
4  16       41           6


In [212]:
series_contagem = []

contador_por_id = {}

for indice, linha in df.iterrows():
    id_atual = linha['ID']

    # Se o id atual não existir no dicionário, inicializa o contador para 1
    if id_atual not in contador_por_id:
        contador_por_id[id_atual] = 1
    else:
        # Caso contrário, incrementa o contador
        contador_por_id[id_atual] += 1

    # Adiciona o valor do contador à lista
    series_contagem.append(contador_por_id[id_atual])


df['Tempo'] = series_contagem

print(df)

       ID  Posicao  Velocidade  Tempo
0      16        0           0      1
1      16       30           1      2
2      16       31           3      3
3      16       35           5      4
4      16       41           6      5
...   ...      ...         ...    ...
2976  496      524          19     31
2977  496      543          18     32
2978  496      562          20     33
2979  496      582          20     34
2980  496      603          20     35

[2981 rows x 4 columns]


In [213]:
correlation_matrix = df[['Tempo', 'Velocidade', 'Posicao']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlação Média (Tempo, Velocidade, Posição)')
plt.show()

In [214]:
X = df[['Tempo', 'Velocidade']]
y = df['Posicao']

In [215]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [216]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [217]:
regressor = KNeighborsRegressor(n_neighbors=3)
regressor.fit(X_train_scaled, y_train)

KNeighborsRegressor(n_neighbors=3)

In [218]:
y_pred = regressor.predict(X_test_scaled)

In [219]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# Apresentar as métricas
print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R²: {r2:.2f}')

MAE: 7.61
MSE: 136.15
RMSE: 11.67
R²: 1.00


In [220]:
error = []
for i in range(1, len(X_train)):  # Teste de 1 a len(X_train) - 1
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    mae = mean_absolute_error(y_test, pred_i)
    error.append(mae)

# Plotando o gráfico de MAE vs. Número de Vizinhos
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(X_train)), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('MAE vs. Número de Vizinhos (K)')
plt.xlabel('Número de Vizinhos (K)')
plt.ylabel('Mean Absolute Error (MAE)')
plt.grid(True)
plt.show()

In [221]:
min_mae = min(error)
best_k = np.array(error).argmin() + 1  # +1 porque o índice começa em 0, mas k começa em 1

print(f'Menor MAE encontrado: {min_mae}')
print(f'Melhor valor de k: {best_k}')

Menor MAE encontrado: 7.248665425201738
Melhor valor de k: 9


In [222]:
best_knn_reg = KNeighborsRegressor(n_neighbors=best_k)
best_knn_reg.fit(X_train, y_train)
y_pred_best = best_knn_reg.predict(X_test)

# Avaliando o desempenho do modelo com o melhor k
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
r2_best = best_knn_reg.score(X_test, y_test)

print(f'\nDesempenho do modelo com o melhor k:')
print(f'r2: {r2_best}')
print(f'mae: {mae_best}')
print(f'mse: {mse_best}')
print(f'rmse: {rmse_best}')


Desempenho do modelo com o melhor k:
r2: 0.9962840412812207
mae: 7.248665425201738
mse: 127.2327194978964
rmse: 11.279748201883605


In [223]:
regressor_knn = KNeighborsRegressor(n_neighbors=9)

In [224]:
regressor_knn.fit(X_train, y_train)

# Fazendo previsões com os dados de teste
y_pred = regressor_knn.predict(X_test)

# Comparando valores reais e preditos
resultado = pd.DataFrame({'Atual': y_test, 'Predicao': y_pred})
print("\nComparação entre valores reais e preditos:")
print(resultado)


Comparação entre valores reais e preditos:
      Atual    Predicao
2408    598  582.333333
1472    223  220.777778
881      30   30.000000
331      36   36.111111
1835    451  423.444444
...     ...         ...
2523     31   31.222222
1811     47   50.333333
1288    277  278.111111
2611    304  292.888889
297      36   36.111111

[895 rows x 2 columns]
